# Get all stop stations within bounding box

In [1]:
import requests

In [2]:
%%time
mapzen_api_key = "mapzen-ai1duha"
bbox = "-122.699432,37.665070,-122.211227,37.861302"
response = requests.get("http://transit.land/api/v1/stop_stations?bbox={}&api_key={}&per_page=false&total=true".format(bbox, mapzen_api_key))
data = response.json()
print data['meta']['total'], "stop_stations in bounding box"
print len(data['stop_stations']), "stop_stations in json response"

5669 stop_stations in bounding box
5669 stop_stations in json response
CPU times: user 403 ms, sys: 67.4 ms, total: 470 ms
Wall time: 56.3 s


# Classify each stop station as either "simple stop" or "station"

In [3]:
def classify_stop_station_types(json_input):
    # May be easier to add this information into json itself rather than separate array...?
    stop_station_types_output = []
    for stop_station in json_input['stop_stations']:
        generated_or_not_generated = []
        
        for stop_egress in stop_station['stop_egresses']:
            generated_or_not_generated.append(stop_egress['generated'])

        for stop_platform in stop_station['stop_platforms']:
            generated_or_not_generated.append(stop_platform['generated'])

        # if a particular station has all stop egresses and stop platforms with
        # generated = true, then it's just a simple stop. otherwise, its a station
        
        if all(generated_or_not_generated):
            stop_station_types_output.append("Simple Stop")
        else:
            stop_station_types_output.append("Station")
            
    return stop_station_types_output

In [4]:
stop_station_types = classify_stop_station_types(data)

print sum((i == "Simple Stop") for i in stop_station_types), "Simple Stops"
print sum((i == "Station") for i in stop_station_types), "Stations"

5649 Simple Stops
20 Stations


# Get all of the stop stations for mapping

In [17]:
def get_stop_stations_for_map(json_input, stop_station_types):
    output = []
    count = 0
    for i in json_input['stop_stations']:
        results = {}
        results['stop_station_lon'] = i['geometry']['coordinates'][0]
        results['stop_station_lat'] = i['geometry']['coordinates'][1]
        results['stop_station_name'] = str(i['name']).replace('"',"")
        results['operators_serving_stop_and_platforms'] = [str(j['operator_name']) for j in i['operators_serving_stop_and_platforms']]
        
        if stop_station_types[count] == 'Station':
            if len(i['stop_platforms']) == 0:
                results['stop_station_type'] = 'Station w 0 Platforms'
            elif len(i['stop_platforms']) == 1:
                results['stop_station_type'] = 'Station w 1 Platform'
            elif len(i['stop_platforms']) == 2:
                results['stop_station_type'] = 'Station w 2 Platforms'
            elif len(i['stop_platforms']) > 2:
                results['stop_station_type'] = 'Station w >2 Platforms'            
        else:        
            results['stop_station_type'] = stop_station_types[count]

        results['number_of_stop_platforms'] = len(i['stop_platforms'])             
                
        output.append(results)
        count += 1
    
    return output

In [18]:
stop_stations = get_stop_stations_for_map(data, stop_station_types)

# Get all of the stop platforms for mapping

In [19]:
def get_stop_platforms_for_map(json_input):
    output = []
    for i in json_input['stop_stations']:
        for stop_platform in i['stop_platforms']:
            platform = {}
            if stop_platform['generated'] == False:
                platform['parent_station_id'] = i['onestop_id']
                platform['parent_station_name'] = i['name']
                platform['stop_platform_lon'] = stop_platform['geometry']['coordinates'][0]
                platform['stop_platform_lat'] = stop_platform['geometry']['coordinates'][1]
                platform['operators_serving_stop_and_platforms'] = i['operators_serving_stop_and_platforms']
                platform['stop_station_type'] = "Stop Platform"
                output.append(platform)                    
    return output

In [20]:
stop_platforms = get_stop_platforms_for_map(data)

# Draw lines from station stops to stop platforms

In [21]:
def get_lines_for_map(json_input):
    lines = []
    for stop_station in json_input['stop_stations']:
        for stop_platform in stop_station['stop_platforms']:
            if stop_platform['generated'] == False:
                stop_station_lon = stop_station['geometry']['coordinates'][0]
                stop_station_lat = stop_station['geometry']['coordinates'][1]
                stop_platform_lon = stop_platform['geometry']['coordinates'][0]
                stop_platform_lat = stop_platform['geometry']['coordinates'][1]
                line = [(stop_station_lat, stop_station_lon), (stop_platform_lat, stop_platform_lon)]
                lines.append(line)
                
    return lines

In [22]:
lines = get_lines_for_map(data)

# Throw it all on a map!

In [11]:
import folium

In [23]:
def make_map(stop_stations, stop_platforms, lines, bbox, basemap = 'Mapbox', save_to_html=False):
    
    # Calculate center location from bbox
    bbox_lons = [float(bbox.split(",")[0]), float(bbox.split(",")[2])]
    bbox_lats = [float(bbox.split(",")[1]), float(bbox.split(",")[3])]
    default_lon = sum(bbox_lons) / len(bbox_lons)
    default_lat = sum(bbox_lats) / len(bbox_lats)
    
    # Choose basemap (currently either Mapbox or OSM)
    if basemap == 'Mapbox':
        baseurl = 'http://{s}.tiles.mapbox.com/v4/mapbox.light/{z}/{x}/{y}.png'
        mapbox_api_key = 'pk.eyJ1Ijoid2lsbGdlYXJ5IiwiYSI6ImNpdW9wcmkxNjAxbDUydXQ0MzFwdmdvOWkifQ.9YIdQhYTOpRpocyFK-tBNA'
        token = '?access_token={}'.format(mapbox_api_key)
        m = folium.Map(location=[default_lat, default_lon], zoom_start=13, tiles=baseurl+token, attr='Mapbox')

    elif basemap == 'OSM':
        m = folium.Map(location=[default_lat, default_lon], zoom_start=13)
    
    # Define point color scheme
    colors = {'Simple Stop': '#3186cc', 
              'Station w 0 Platforms': '#fee5d9',
              'Station w 1 Platform': '#fc9272', 
              'Station w 2 Platforms': '#ef3b2c',
              'Station w >2 Platforms': '#99000d', 
              'Stop Platform': '#32CD32', 
              'Line': '#000000'}
    
    # Make radius bigger for stations with more platforms
    station_radius = {'Station w 0 Platforms': 3,
                      'Station w 1 Platform': 4,
                      'Station w 2 Platforms': 5,
                      'Station w >2 Platforms': 6,
                      'Simple Stop': 3}
    # Draw lines
    for i in lines:
        folium.PolyLine(locations=i, color=colors['Line'], weight=2, opacity=1).add_to(m)
        
    # Draw stop_platforms
    for i in stop_platforms:
        popup_html = folium.Html('<b>Name: </b> {} <br> <b>Operators:</b> {} <br> <b>Type:</b> {}'.format(i['parent_station_name'].replace("'",""), i['operators_serving_stop_and_platforms'][0]['operator_name'], i['stop_station_type']), script=True)
        popup = folium.Popup(popup_html, max_width=2650)
        folium.CircleMarker(location=[i['stop_platform_lat'], i['stop_platform_lon']], radius=4,
                       popup=popup, color=colors[i['stop_station_type']], fill_opacity=0.8,
                       fill_color=colors[i['stop_station_type']]).add_to(m)
        
    # Draw station_stops
    for i in station_stops:
        popup_html = folium.Html('<b>Name: </b> {} <br> <b>Operators:</b> {} <br> <b>Type:</b> {}'.format(i['stop_station_name'].replace("'",""), ','.join(i['operators_serving_stop_and_platforms']), i['stop_station_type']), script=True)
        popup = folium.Popup(popup_html, max_width=2650)
        folium.CircleMarker(location=[i['stop_station_lat'], i['stop_station_lon']], radius=station_radius[i['stop_station_type']],
                        popup=popup, color=colors[i['stop_station_type']], fill_opacity=0.8,
                        fill_color=colors[i['stop_station_type']]).add_to(m)

    # Optional save map to html
    if save_to_html:
        m.save(save_to_html)
    else:
        return m

In [13]:
%%time
make_map(stop_stations, stop_platforms, lines, bbox, save_to_html="../maps/map5.html")

CPU times: user 4min 14s, sys: 3.54 s, total: 4min 17s
Wall time: 4min 23s


# "Fuzzy string matching" to detect simple stops that should be platforms

In [ ]:
def get_stop_stations(json_input, stop_station_types):
    output = []
    count = 0
    for i in json_input['stop_stations']:
        results = {}
        results['stop_station_lon'] = i['geometry']['coordinates'][0]
        results['stop_station_lat'] = i['geometry']['coordinates'][1]
        results['stop_station_name'] = str(i['name']).replace('"',"")
        results['operators_serving_stop_and_platforms'] = [str(j['operator_name']) for j in i['operators_serving_stop_and_platforms']]
        
        if stop_station_types[count] == 'Station':
            if len(i['stop_platforms']) == 0:
                results['stop_station_type'] = 'Station w 0 Platforms'
            elif len(i['stop_platforms']) == 1:
                results['stop_station_type'] = 'Station w 1 Platform'
            elif len(i['stop_platforms']) == 2:
                results['stop_station_type'] = 'Station w 2 Platforms'
            elif len(i['stop_platforms']) > 2:
                results['stop_station_type'] = 'Station w >2 Platforms'            
        else:        
            results['stop_station_type'] = stop_station_types[count]

        results['number_of_stop_platforms'] = len(i['stop_platforms'])             
                
        output.append(results)
        count += 1
    
    return output

In [31]:
data['stop_stations'][0]

{u'created_at': u'2016-02-06T20:08:39.838Z',
 u'created_or_updated_in_changeset_id': 6383,
 u'geometry': {u'coordinates': [-122.251371, 37.844702], u'type': u'Point'},
 u'name': u'Rockridge',
 u'onestop_id': u's-9q9p616uyb-rockridge',
 u'operators_serving_stop_and_platforms': [{u'operator_name': u'Bay Area Rapid Transit',
   u'operator_onestop_id': u'o-9q9-bart'}],
 u'routes_serving_stop_and_platforms': [{u'operator_name': u'Bay Area Rapid Transit',
   u'operator_onestop_id': u'o-9q9-bart',
   u'route_name': u'Pittsburg/Bay Point - SFIA/Millbrae',
   u'route_onestop_id': u'r-9q9-pittsburg~baypoint~sfia~millbrae'}],
 u'stop_egresses': [{u'created_at': None,
   u'created_or_updated_in_changeset_id': None,
   u'directionality': None,
   u'generated': True,
   u'geometry': {u'coordinates': [-122.251371, 37.844702], u'type': u'Point'},
   u'last_conflated_at': u'2017-06-21T02:17:25.087Z',
   u'name': u'Rockridge',
   u'onestop_id': u's-9q9p616uyb-rockridge>',
   u'osm_way_id': 53827767,
   

In [15]:
len(stop_platforms)

88

In [25]:
len(stop_stations)

5669

In [26]:
len(stop_station_types)

5669

In [27]:
len(lines)

88

In [28]:
stop_stations[0]

{'number_of_stop_platforms': 1,
 'operators_serving_stop_and_platforms': ['Bay Area Rapid Transit'],
 'stop_station_lat': 37.844702,
 'stop_station_lon': -122.251371,
 'stop_station_name': 'Rockridge',
 'stop_station_type': 'Simple Stop'}

In [ ]:
['onestop_id',
 'routes_serving_stop_and_platforms',
 'stop_egresses',
 'stop_platforms',
 


In [30]:
data['stop_stations'][0]

{u'created_at': u'2016-02-06T20:08:39.838Z',
 u'created_or_updated_in_changeset_id': 6383,
 u'geometry': {u'coordinates': [-122.251371, 37.844702], u'type': u'Point'},
 u'name': u'Rockridge',
 u'onestop_id': u's-9q9p616uyb-rockridge',
 u'operators_serving_stop_and_platforms': [{u'operator_name': u'Bay Area Rapid Transit',
   u'operator_onestop_id': u'o-9q9-bart'}],
 u'routes_serving_stop_and_platforms': [{u'operator_name': u'Bay Area Rapid Transit',
   u'operator_onestop_id': u'o-9q9-bart',
   u'route_name': u'Pittsburg/Bay Point - SFIA/Millbrae',
   u'route_onestop_id': u'r-9q9-pittsburg~baypoint~sfia~millbrae'}],
 u'stop_egresses': [{u'created_at': None,
   u'created_or_updated_in_changeset_id': None,
   u'directionality': None,
   u'generated': True,
   u'geometry': {u'coordinates': [-122.251371, 37.844702], u'type': u'Point'},
   u'last_conflated_at': u'2017-06-21T02:17:25.087Z',
   u'name': u'Rockridge',
   u'onestop_id': u's-9q9p616uyb-rockridge>',
   u'osm_way_id': 53827767,
   